# Import csv files

## create .env

In [1]:
!pip install python-dotenv

In [7]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Test if variables are loaded
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_port = os.getenv('DB_PORT')
database_url = os.getenv("DATABASE_URL")
secret_key = os.getenv("SECRET_KEY")
debug_mode = os.getenv("DEBUG")

# print("✓ Environment variables loaded:")
# print(f"DB_HOST: {os.getenv('DB_HOST')}")
# print(f"DB_NAME: {os.getenv('DB_NAME')}")
# print(f"DB_USER: {os.getenv('DB_USER')}")
# print(f"DB_PASSWORD: {os.getenv('DB_PASSWORD')}")
# print(f"DB_PORT: {os.getenv('DB_PORT')}")
# print(f"Database URL: {database_url}")
# print(f"Secret Key: {secret_key}")
# print(f"Debug Mode: {debug_mode}")

## Create Tables and Import Data Using Python

In [8]:
import psycopg2
import pandas as pd
from psycopg2 import sql

conn_params = {
    'host':     db_host,
    'database': db_name,
    'user':     db_user,
    'password': db_password,
    'port':     db_port
}

try:
    conn = psycopg2.connect(**conn_params)
    conn.autocommit = True
    cursor = conn.cursor()
    # Create database
    cursor.execute("CREATE DATABASE final_project;")
    print("Database created successfully!")
    
except psycopg2.errors.DuplicateDatabase:
    print("Database already exists")

except Exception as e:
    print(f"Error: {e}")

finally:
    cursor.close()
    conn.close()

Database already exists


## Create Tables from Your CSV Files

In [9]:
# Connect to your project database
conn_params['database'] = os.getenv('DB_NAME')

try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    
    # Create table with proper data types
    create_table_query = """
    CREATE TABLE IF NOT EXISTS supply_chain_df (
        type VARCHAR(50),
        days_for_shipping_real INTEGER,
        days_for_shipment_scheduled INTEGER,
        benefit_per_order NUMERIC(10,2),
        sales_per_customer NUMERIC(10,2),
        delivery_status VARCHAR(50),
        late_delivery_risk INTEGER,
        category_id INTEGER,
        category_name VARCHAR(100),
        customer_city VARCHAR(100),
        customer_country VARCHAR(100),
        customer_email VARCHAR(150),
        customer_fname VARCHAR(100),
        customer_id INTEGER,
        customer_lname VARCHAR(100),
        customer_password VARCHAR(100),
        customer_segment VARCHAR(50),
        customer_state VARCHAR(100),
        customer_street VARCHAR(200),
        customer_zipcode VARCHAR(20),
        department_id INTEGER,
        department_name VARCHAR(100),
        latitude NUMERIC(10,6),
        longitude NUMERIC(10,6),
        market VARCHAR(50),
        order_city VARCHAR(100),
        order_country VARCHAR(100),
        order_customer_id INTEGER,
        order_date DATE,
        order_id INTEGER PRIMARY KEY,
        order_item_cardprod_id INTEGER,
        order_item_discount NUMERIC(10,2),
        order_item_discount_rate NUMERIC(5,4),
        order_item_id INTEGER,
        order_item_product_price NUMERIC(10,2),
        order_item_profit_ratio NUMERIC(5,4),
        order_item_quantity INTEGER,
        sales NUMERIC(10,2),
        order_item_total NUMERIC(10,2),
        order_profit_per_order NUMERIC(10,2),
        order_region VARCHAR(50),
        order_state VARCHAR(100),
        order_status VARCHAR(50),
        order_zipcode VARCHAR(20),
        product_card_id INTEGER,
        product_category_id INTEGER,
        product_description TEXT,
        product_image VARCHAR(200),
        product_name VARCHAR(200),
        product_price NUMERIC(10,2),
        product_status INTEGER,
        shipping_date DATE,
        shipping_mode VARCHAR(50)
    );
    """
    
    cursor.execute(create_table_query)
    conn.commit()
    print("✓ Table created successfully!")
    
except Exception as e:
    print(f"Error: {e}")
    conn.rollback()
    
finally:
    cursor.close()
    conn.close()


✓ Table created successfully!


## Import DataCoSupplyChainDataset


In [10]:
from sqlalchemy import create_engine

db_url = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(db_url)

supply_chain_df = pd.read_csv('resources/DataCoSupplyChainDataset.csv')
supply_chain_df.columns = supply_chain_df.columns.str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
supply_chain_df.to_sql('supply_chain_df', engine, if_exists='replace', index=False)

print(f"✓ Successfully imported {len(supply_chain_df)} rows!")

✓ Successfully imported 180519 rows!


## Import tokenized_access_log_df

In [11]:
# Connection parameters

try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    
    # CREATE TABLE SQL statement
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS access_log_df (
        product VARCHAR(200),
        category VARCHAR(100),
        date DATE,
        month VARCHAR(20),
        hour TIME,
        department VARCHAR(100),
        ip VARCHAR(50),
        url TEXT
    );
    """
    
    cursor.execute(create_table_sql)
    conn.commit()
    print("✓ Table created successfully!")
    
except Exception as e:
    print(f"Error: {e}")
    conn.rollback()
finally:
    cursor.close()
    conn.close()


✓ Table created successfully!


In [13]:

access_log_df = pd.read_csv('resources/tokenized_access_logs.csv')
access_log_df.columns = access_log_df.columns.str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
access_log_df.to_sql('access_log_df', engine, if_exists='replace', index=False)

print(f"✓ Successfully imported {len(access_log_df)} rows!")

✓ Successfully imported 469977 rows!


In [ ]:
supply_chain_df.head(5)

,type,days_for_shipping_real,days_for_shipment_scheduled,benefit_per_order,sales_per_customer,delivery_status,late_delivery_risk,category_id,category_name,customer_city,...,order_zipcode,product_card_id,product_category_id,product_description,product_image,product_name,product_price,product_status,shipping_date_dateorders,shipping_mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [ ]:
access_log_df.head(5)

,product,category,date,month,hour,department,ip,url
0,adidas Brazuca 2017 Official Match Ball,baseball & softball,9/1/2017 6:00,Sep,6,fitness,37.97.182.65,/department/fitness/category/baseball%20&%20so...
1,The North Face Women's Recon Backpack,hunting & shooting,9/1/2017 6:00,Sep,6,fan shop,206.56.112.1,/department/fan%20shop/category/hunting%20&%20...
2,adidas Kids' RG III Mid Football Cleat,featured shops,9/1/2017 6:00,Sep,6,apparel,215.143.180.0,/department/apparel/category/featured%20shops/...
3,Under Armour Men's Compression EV SL Slide,electronics,9/1/2017 6:00,Sep,6,footwear,206.56.112.1,/department/footwear/category/electronics/prod...
4,Pelican Sunstream 100 Kayak,water sports,9/1/2017 6:01,Sep,6,fan shop,136.108.56.242,/department/fan%20shop/category/water%20sports...


# Write SQL Queries

In [ ]:
conn = psycopg2.connect(**conn_params)

def query(sql, conn):
    try:
        # conn = psycopg2.connect(**conn_params)
        df = pd.read_sql(sql, conn)
        conn.close()
        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
df = query("SELECT * FROM supply_chain_df LIMIT 10;",conn)
df

/var/folders/yt/wnnkxst90pz1w0b5ww0xhh6r0000gn/T/ipykernel_63648/2644468717.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,type,days_for_shipping_real,days_for_shipment_scheduled,benefit_per_order,sales_per_customer,delivery_status,late_delivery_risk,category_id,category_name,customer_city,...,order_zipcode,product_card_id,product_category_id,product_description,product_image,product_name,product_price,product_status,shipping_date_dateorders,shipping_mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,None,1360,73,None,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,None,1360,73,None,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,None,1360,73,None,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,None,1360,73,None,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,None,1360,73,None,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class
5,TRANSFER,6,4,18.580000,294.980011,Shipping canceled,0,73,Sporting Goods,Tonawanda,...,None,1360,73,None,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/19/2018 11:03,Standard Class
6,DEBIT,2,1,95.180000,288.420013,Late delivery,1,73,Sporting Goods,Caguas,...,None,1360,73,None,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 10:42,First Class
7,TRANSFER,2,1,68.430000,285.140015,Late delivery,1,73,Sporting Goods,Miami,...,None,1360,73,None,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 10:21,First Class
8,CASH,3,2,133.720001,278.589996,Late delivery,1,73,Sporting Goods,Caguas,...,None,1360,73,None,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 10:00,Second Class
9,CASH,2,1,132.149994,275.309998,Late delivery,1,73,Sporting Goods,San Ramon,...,None,1360,73,None,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 9:39,First Class
